# Obtener el primera fecha por canal y cluster

## Carga de librerías necesarias

In [1]:
from pymongo import MongoClient
from dotenv import load_dotenv
from ast import literal_eval
from tqdm.auto import tqdm
from motor.motor_asyncio import AsyncIOMotorClient
from tqdm.asyncio import tqdm_asyncio  # barra de progreso asíncrona

import os
import asyncio
import pandas as pd

## Conexión BD

In [2]:
# Cargar las variables desde el archivo .env.mongo
load_dotenv(dotenv_path="../.env.mongo")

# Leer las variables de entorno
user = os.getenv("MONGO_USER")
password = os.getenv("MONGO_PASSWORD")
host = os.getenv("MONGO_HOST")
port = os.getenv("MONGO_PORT")

# Cliente de conexión a MongoDB
mongo_url = f"mongodb://{user}:{password}@{host}:{port}/?authSource=admin"
client = AsyncIOMotorClient(mongo_url)

# Verificar conexión
try:
    client.admin.command("ping")
    print("Conexión exitosa a MongoDB")

except Exception as e:
    print("Error de conexión:", e)

Conexión exitosa a MongoDB


In [3]:
db = client["tg_channels_cositas"]

## Carga de datos de macro-tópicos

In [31]:
macrot_df = pd.read_csv("../data/macro_topics.csv")
macrot_df.head().iloc[:, 1:]

,tid,n_messages,description,keywords,representative_docs,theme,rep_docs_norm,representative_docs_str,macrot_id,cluster,cluster_words,cluster_words_list
0,0,111,0_vacunamos_novavax_fauci_reacciones,"['vacunamos', 'novavax', 'fauci', 'reacciones'...",['estracto de la agencia europea del medicamen...,Conspiracy theories and misinformation about C...,['estracto de la agencia europea del medicamen...,estracto de la agencia europea del medicamento...,111,0,deaths | incl | desconocido | casos | disorders,"['deaths', 'incl', 'desconocido', 'casos', 'di..."
1,1,106,1_encontrado_oficialialistas_mujer_desconectado,"['encontrado', 'oficialialistas', 'mujer', 'de...","['los viajes espaciales, la creacion de una pa...",The theme revolves around conspiracy theories ...,['los viajes espaciales la creacion de una pan...,los viajes espaciales la creacion de una pande...,-1,-1,NaN,[]
2,2,85,2_expreso0demedianoche_directo_anonimo_descarg...,"['expreso0demedianoche', 'directo', 'anonimo',...",['el expreso de media noche directo del 24/02 ...,Unauthorized use of a popular podcast's name a...,['el expreso de media noche directo del 24 02 ...,el expreso de media noche directo del 24 02 21...,-1,-1,NaN,[]
3,3,65,3_despiertos_actuamos_inocencia_hijos,"['despiertos', 'actuamos', 'inocencia', 'hijos...",['da mucho asco ver ciertos personajes que van...,"Critique of egoism, superficiality, and lack o...",['da mucho asco ver ciertos personajes que van...,da mucho asco ver ciertos personajes que van d...,80,0,doktor | esra | medical | gmc | dr,"['doktor', 'esra', 'medical', 'gmc', 'dr']"
4,4,63,4_protestas_republica_belga_cerrados,"['protestas', 'republica', 'belga', 'cerrados'...",['la gente se defiende: suiza fuerza un refere...,Public discontent and resistance against gover...,['la gente se defiende suiza fuerza un referen...,la gente se defiende suiza fuerza un referendu...,64,8,francia | france | macron | sam | french,"['francia', 'france', 'macron', 'sam', 'french']"


In [32]:
print(macrot_df.shape)

(21614, 13)


In [33]:
macrot_df = macrot_df[macrot_df["cluster"] != -1]
print(macrot_df.shape)

(9542, 13)


In [8]:
# dict {(_id, cluster): [tid1, tid2, …]}
col_tids = (
    macrot_df
    .groupby(['_id', 'cluster'])['tid']
    .unique()
    .to_dict()
)

async def procesar_coleccion(collection_name, cluster_id, tids, batch_size=100):
    coll = db[collection_name]
    out = []
    for i in range(0, len(tids), batch_size):
        batch = [int(t) for t in tids[i:i+batch_size]]
        pipeline = [
            {'$match': {'topic_id': {'$in': batch}}},
            {'$group': {
                '_id': '$topic_id',
                'earliest_date': {'$min': '$date'}
            }}
        ]
        cursor = coll.aggregate(pipeline, allowDiskUse=True)
        async for doc in cursor:
            out.append({
                '_id': collection_name,
                'cluster':     cluster_id,
                'tid':         doc['_id'],
                'earliest_date': pd.to_datetime(doc['earliest_date'])
            })
    return out

async def main():
    tasks = [
        procesar_coleccion(col_name, cluster, tids)
        for (col_name, cluster), tids in col_tids.items()
    ]
    results = []
    for coro in tqdm_asyncio.as_completed(tasks, total=len(tasks),
                                          desc="Procesando colecciones"):
        results.extend(await coro)
    return results

# Ejecuta el loop
results = await main()

Procesando colecciones: 100%|██████████| 2975/2975 [1:10:23<00:00,  1.42s/it]


In [34]:
# Unión con el df de macro-tópicos
earliest_df = pd.DataFrame(results).drop(columns=["tid"])
macrot_df = macrot_df.merge(earliest_df, on=['_id', 'cluster'], how='left')

In [35]:
macrot_df["earliest_date"].describe()

count                                 156576
mean     2021-09-16 14:09:16.204226816+00:00
min                2015-12-21 16:39:15+00:00
25%                2021-02-25 13:51:53+00:00
50%                2021-09-24 07:52:24+00:00
75%                2022-03-27 14:30:40+00:00
max                2024-02-17 21:39:31+00:00
Name: earliest_date, dtype: object

In [36]:
macrot_df.to_csv("../data/macrot_tiempos.csv", index=False)

In [37]:
df_netinf = macrot_df[["_id", "cluster", "earliest_date"]]
df_netinf = df_netinf.rename(columns={
    'cluster': 'cascade_id',
    'rep_docs_dates': 'event_time',
    '_id': 'node_name'
})
df_netinf.head()

,node_name,cascade_id,earliest_date
0,disidentesreunios,0,2020-11-09 16:16:05+00:00
1,disidentesreunios,0,2020-11-10 19:52:12+00:00
2,disidentesreunios,0,2020-11-06 13:02:32+00:00
3,disidentesreunios,0,2020-11-09 09:09:57+00:00
4,disidentesreunios,0,2020-11-09 16:16:05+00:00


In [38]:
# Ordenar el DataFrame por cascada y tiempo de evento
df_netinf = df_netinf.sort_values(['cascade_id', 'earliest_date'])
df_netinf.head()

,node_name,cascade_id,earliest_date
88345,realDonaldjTrump,0,2017-03-21 13:39:19+00:00
88352,realDonaldjTrump,0,2017-03-21 13:39:19+00:00
88346,realDonaldjTrump,0,2017-06-07 18:40:45+00:00
88353,realDonaldjTrump,0,2017-06-07 18:40:45+00:00
8520,Planet_Earth,0,2017-12-06 13:00:26+00:00


In [39]:
df_netinf = (
    df_netinf
    .groupby(['node_name', 'cascade_id'], as_index=False)
    ['earliest_date']
    .min()
    .sort_values(['cascade_id', 'earliest_date'])
    .reset_index(drop=True)
)
df_netinf.head()

,node_name,cascade_id,earliest_date
0,realDonaldjTrump,0,2017-03-21 13:39:19+00:00
1,Planet_Earth,0,2017-12-06 13:00:26+00:00
2,dokuz8haber,0,2018-05-02 10:08:50+00:00
3,police_frequency,0,2018-05-15 09:13:59+00:00
4,nationalistesfr,0,2018-05-30 06:46:46+00:00


In [40]:
df_netinf.shape

(2975, 3)

In [41]:
df_netinf.to_csv("../data/netinf_data.csv", index=False)